<a href="https://colab.research.google.com/github/Janani-SB/GenAI_projects-/blob/main/Mini_Project_1_Review_Analyst.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project: Prompt Engineering with LangChain and ChatGPT for real-world tasks

In this notebook you will leverage ChatGPT and LangChain to solve and do a few mini-projects based on some real-world scenarios:

- Mini-Project 1: Review Analyst
- Mini-Project 2: Research Paper Analyst
- Mini-Project 3: Social Media Marketing Analyst
- Mini-Project 4: IT Support Analyst


# **Mini-Project 1: Review Analyst**

You are building an AI system to be able to look at customer reviews and do some complex analysis. for each review get ChatGPT to do the following:

-- Summarize the review. The summary should be at most 3 lines.
-- Highlight both the positives and negatives
-- Display the overall sentiment of the review (positive, negative, neutral)
-- Display a list of 3 - 5 emotions expressed by the customer in the review
-- If the sentiment is positive or neutral
 write an email and thank them for the review
--If the sentiment is negative apologize and write an email with an appropriate response
--Try to get the response in a nice structured format using an output parser

#Tech stack summary
•	Used LangChain, PydanticOutputParser, and OpenAI ChatGPT to summarize customer reviews, extract pros/cons, emotions, and sentiment.
•	Implemented conditional email generation based on sentiment analysis using LCEL chaining.
•	Structured the entire response pipeline using PromptTemplate → LLM → Structured Output.


In [ ]:
!pip install langchain
!pip install langchain-openai
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00


In [ ]:
# Enter the API Tokens
from getpass import getpass
OPENAI_KEY = getpass("Please enter your OPEN AI API Key here :")

Please enter your OPEN AI API Key here :··········


In [ ]:
import os
os.environ["OPENAI_API_KEY"] = OPENAI_KEY

In [ ]:
from langchain_openai import ChatOpenAI
chatgpt = ChatOpenAI(model_name="gpt-4", temperature=0.9)

#Access Customer Reviews

In [ ]:
reviews = [
    f"""
    Just received the Bluetooth speaker I ordered for beach outings, and it's fantastic.
    The sound quality is impressively clear with just the right amount of bass.
    It's also waterproof, which tested true during a recent splashing incident.
    Though it's compact, the volume can really fill the space.
    The price was a bargain for such high-quality sound.
    Shipping was also on point, arriving two days early in secure packaging.
    """,
    f"""
    Purchased a new gaming keyboard because of its rave reviews about responsiveness and backlighting.
    It hasn't disappointed. The keys have a satisfying click and the LED colors are vibrant,
    enhancing my gaming experience significantly. Price-wise, it's quite competitive,
    and I feel like I got a good deal. The delivery was swift, and it came well-protected,
    ensuring no damage during transport.
    """,
    f"""
    Ordered a set of wireless earbuds for running, and they've been a letdown.
    The sound constantly cuts out, and the fit is uncomfortable after only a few minutes of use.
    They advertised a 12-hour battery life, but I'm barely getting four hours.
    Considering the cost, I expected better quality and performance.
    They did arrive on time, but the positives end there. I'm already looking into a return.
    """,
    f"""
    The tablet stand I bought was touted as being sturdy and adjustable,
    but it's anything but. It wobbles with the slightest touch,
    and the angles are not holding up as promised. It feels like a breeze could knock it over.
    It was also pricier than others I've seen, which adds to the disappointment.
    It did arrive promptly, but what's the use if the product doesn't meet basic expectations?
    """,
    f"""
    Needed a new kitchen blender, but this model has been a nightmare.
    It's supposed to handle various foods, but it struggles with anything tougher than cooked vegetables.
    It's also incredibly noisy, and the 'easy-clean' feature is a joke; food gets stuck under the blades constantly.
    I thought the brand meant quality, but this product has proven me wrong.
    Plus, it arrived three days late. Definitely not worth the expense.
    """
]

# Define Output Parser basis the problem statement

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

class ReviewAnalysisResponse(BaseModel):
    summary: str = Field(description="A brief summary of the customer review with maximum 3 lines.")
    positive: list = Field(description="A list showing the positives mentioned by the customer in the review if any - max 3 points")
    negative: list = Field(description="A list showing the negatives mentioned by the customer in the review if any - max 3 points")
    sentiment: str = Field(description="One word showing the sentiment of the review - positive, negative or neutral")
    emotions: list = Field(description="A list of 3 - 5 emotions expressed by the customer in the review")
    email: str = Field(description="Detailed email to the customer based on the sentiment")

parser = PydanticOutputParser(pydantic_object=ReviewAnalysisResponse)
parser

PydanticOutputParser(pydantic_object=<class '__main__.ReviewAnalysisResponse'>)

# Create the input prompt for the LLM

In [ ]:
prompt_txt = """
Analyze the given customer review below and generate the response based on the instructions
mentioned below in the format instruction.
Also remember to write a detailed email response for the email field based on these conditions:
     - email should be addressed to Dear Customer and signed with Service Agent
     - thank them if the review is positive or neutral
     - apologize if the review is negative

     Format Instructions: {format_instructions}

     Review: {review}
"""

prompt = PromptTemplate(template = prompt_txt, input_variables = ["review"],
                        partial_variables = {"format_instructions":parser.get_format_instructions()},)

In [ ]:
prompt

PromptTemplate(input_variables=['review'], input_types={}, partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"summary": {"description": "A brief summary of the customer review with maximum 3 lines.", "title": "Summary", "type": "string"}, "positive": {"description": "A list showing the positives mentioned by the customer in the review if any - max 3 points", "items": {}, "title": "Positive", "type": "array"}, "negative": {"description": "A list showing the negatives mentioned by the customer in the review if any - max 3 points"

# Create a LCEL LLM Chain

In [ ]:
chain = (prompt | chatgpt | parser)

# Format the input review

In [ ]:
reviews_formatted = [{"review": review} for review in reviews]
reviews_formatted

[{'review': "\n    Just received the Bluetooth speaker I ordered for beach outings, and it's fantastic.\n    The sound quality is impressively clear with just the right amount of bass.\n    It's also waterproof, which tested true during a recent splashing incident.\n    Though it's compact, the volume can really fill the space.\n    The price was a bargain for such high-quality sound.\n    Shipping was also on point, arriving two days early in secure packaging.\n    "},
 {'review': "\n    Purchased a new gaming keyboard because of its rave reviews about responsiveness and backlighting.\n    It hasn't disappointed. The keys have a satisfying click and the LED colors are vibrant,\n    enhancing my gaming experience significantly. Price-wise, it's quite competitive,\n    and I feel like I got a good deal. The delivery was swift, and it came well-protected,\n    ensuring no damage during transport.\n    "},
 {'review': "\n    Ordered a set of wireless earbuds for running, and they've been 

# Get responses from the LLM

In [ ]:
responses = chain.map().invoke(reviews_formatted)
responses

[ReviewAnalysisResponse(summary='The customer is satisfied with the Bluetooth speaker ordered, praising the sound quality, bass, waterproof feature, compactness, volume, and price. The speaker also arrived early in secure packaging.', positive=['Impressive sound quality', 'Waterproof feature', 'Arrived early in secure packaging'], negative=[], sentiment='positive', emotions=['satisfied', 'impressed', 'happy'], email='Dear Customer,\n\nThank you for your positive review. We are delighted to hear that you are satisfied with the Bluetooth speaker, its sound quality, and its waterproof feature. We are also glad that the product reached you earlier than expected and in secure packaging. We strive to provide our customers with high-quality products and efficient shipping services.\n\nThank you once again for your kind words. We look forward to serving you again.\n\nBest regards,\nService Agent'),
 ReviewAnalysisResponse(summary='Customer is pleased with the gaming keyboard, praising its resp

In [ ]:
for response in responses:
    for k, v in response.dict().items():
        print(f'{k}:\n{v}')
    print("***********************************************")
    print('\n')

summary:
Customer is satisfied with the Bluetooth speaker they ordered for beach outings, impressed by the sound quality, the bass, the waterproof feature and the volume of the speaker.
positive:
['Sound quality is impressively clear', 'Just the right amount of bass', 'Waterproof feature tested true', 'Compact but has high volume', 'Price was a bargain', 'Shipping was quick and secure']
negative:
[]
sentiment:
positive
emotions:
['satisfaction', 'impressed', 'happiness', 'relief', 'excitement']
email:
Dear Customer, 

Thank you for taking the time to review our product. We are thrilled to hear that you're satisfied with the Bluetooth speaker and that it met your expectations in terms of sound quality, bass, compactness, and price. We also appreciate your positive feedback regarding our shipping service. We continuously strive to serve our customers better. Thank you once again for your positive review. 

Best Regards, 
Service Agent
***********************************************


sum

<ipython-input-19-1459258382>:2: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  for k, v in response.dict().items():
